In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/drw-crypto-market-prediction/sample_submission.csv
/kaggle/input/drw-crypto-market-prediction/train.parquet
/kaggle/input/drw-crypto-market-prediction/test.parquet


In [2]:
from sklearn.model_selection import KFold, train_test_split
from xgboost import XGBRegressor
from scipy.stats import pearsonr
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Deep Learning imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# =========================
# Configuration
# =========================
class Config:
    TRAIN_PATH = "/kaggle/input/drw-crypto-market-prediction/train.parquet"
    TEST_PATH = "/kaggle/input/drw-crypto-market-prediction/test.parquet"
    SUBMISSION_PATH = "/kaggle/input/drw-crypto-market-prediction/sample_submission.csv"

    FEATURES = [
        "X863", "X856", "X598", "X862", "X385", "X852", "X603", "X860", "X674",
        "X415", "X345", "X855", "X174", "X302", "X178", "X168", "X612", "bid_qty",
        "ask_qty", "buy_qty", "sell_qty", "volume", "X888", "X421", "X333","X817", 
        "X586",  "X292"
    ]
    
    # Features for MLP (subset)
    MLP_FEATURES = [
        "X863", "X856", "X344", "X598", "X862", "X385", "X852", "X603", "X860", "X674",
        "X415", "X345", "X137", "X855", "X174", "X302", "X178", "X532", "X168", "X612",
        "bid_qty", "ask_qty", "buy_qty", "sell_qty", "volume"
    ]

    LABEL_COLUMN = "label"
    N_FOLDS = 3
    RANDOM_STATE = 42
    OUTLIER_FRACTION = 0.001  # 0.1% of records
    
    # Outlier handling strategies to test
    OUTLIER_STRATEGIES = ["reduce", "remove", "double", "none"]

XGB_PARAMS = {
    "tree_method": "hist",
    "device": "gpu" if torch.cuda.is_available() else "cpu",
    "colsample_bylevel": 0.4778,
    "colsample_bynode": 0.3628,
    "colsample_bytree": 0.7107,
    "gamma": 1.7095,
    "learning_rate": 0.02213,
    "max_depth": 20,
    "max_leaves": 12,
    "min_child_weight": 16,
    "n_estimators": 1667,
    "subsample": 0.06567,
    "reg_alpha": 39.3524,
    "reg_lambda": 75.4484,
    "verbosity": 0,
    "random_state": Config.RANDOM_STATE,
    "n_jobs": -1
}

LEARNERS = [
    {"name": "xgb", "Estimator": XGBRegressor, "params": XGB_PARAMS}
]

# =========================
# Deep Learning Components
# =========================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def get_activation_function(name):
    """Return the activation function based on the name."""
    if name == None:
        return None
    name = name.lower()
    if name == 'relu':
        return nn.ReLU()
    elif name == 'tanh':
        return nn.Tanh()
    elif name == 'sigmoid':
        return nn.Sigmoid()
    else:
        raise ValueError(f"Unsupported activation function: {name}")

class MLP(nn.Module):
    def __init__(self, dropout_rate=0.6, 
                 layers=[128, 64], activation='relu', last_activation=None):
        super(MLP, self).__init__()
        
        self.linears = nn.ModuleList()
        self.activation = get_activation_function(activation)
        self.last_activation = get_activation_function(last_activation)

        for i in range(len(layers) - 1):
            self.linears.append(nn.Linear(layers[i], layers[i + 1]))

        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        for k in range(len(self.linears) - 1):
            x = self.activation(self.linears[k](x))
            x = self.dropout(x)
        x = self.linears[-1](x)
        if self.last_activation is not None:
            x = self.last_activation(x)
        return x

class Checkpointer:
    def __init__(self, path="best_model.pt"):
        self.path = path
        self.best_pearson = -np.inf

    def load(self, model):
        """Load the best model weights."""
        model.load_state_dict(torch.load(self.path, map_location=device))
        print(f"Model loaded from {self.path} with best Pearson: {self.best_pearson:.4f}")
        return model

    def __call__(self, pearson_coef, model):
        """Call method to save the model if the Pearson coefficient is better than the best one."""
        if pearson_coef > self.best_pearson:
            self.best_pearson = pearson_coef
            torch.save(model.state_dict(), self.path)
            print(f"✅ New best model saved with Pearson: {pearson_coef:.4f}")

def get_dataloaders(X, Y, hparams, device, shuffle=True):
    """Create DataLoader for training and validation datasets."""
    X_tensor = torch.tensor(X, dtype=torch.float32, device=device)
    if Y is not None:
        Y_tensor = torch.tensor(Y.values if hasattr(Y, 'values') else Y, 
                                dtype=torch.float32, device=device).unsqueeze(1)
        dataset = TensorDataset(X_tensor, Y_tensor)
    else:
        dataset = TensorDataset(X_tensor)
    
    dataloader = DataLoader(dataset, batch_size=hparams["batch_size"], shuffle=shuffle, 
                            generator=torch.Generator().manual_seed(hparams["seed"]))
    return dataloader

# =========================
# Feature Engineering
# =========================
def add_features(df):
    # Original features
    df['bid_ask_interaction'] = df['bid_qty'] * df['ask_qty']
    df['bid_buy_interaction'] = df['bid_qty'] * df['buy_qty']
    df['bid_sell_interaction'] = df['bid_qty'] * df['sell_qty']
    df['ask_buy_interaction'] = df['ask_qty'] * df['buy_qty']
    df['ask_sell_interaction'] = df['ask_qty'] * df['sell_qty']

    df['volume_weighted_sell'] = df['sell_qty'] * df['volume']
    df['buy_sell_ratio'] = df['buy_qty'] / (df['sell_qty'] + 1e-10)
    df['selling_pressure'] = df['sell_qty'] / (df['volume'] + 1e-10)
    df['log_volume'] = np.log1p(df['volume'])

    df['effective_spread_proxy'] = np.abs(df['buy_qty'] - df['sell_qty']) / (df['volume'] + 1e-10)
    df['bid_ask_imbalance'] = (df['bid_qty'] - df['ask_qty']) / (df['bid_qty'] + df['ask_qty'] + 1e-10)
    df['order_flow_imbalance'] = (df['buy_qty'] - df['sell_qty']) / (df['buy_qty'] + df['sell_qty'] + 1e-10)
    df['liquidity_ratio'] = (df['bid_qty'] + df['ask_qty']) / (df['volume'] + 1e-10)
    
    # === NEW MICROSTRUCTURE FEATURES ===
    
    # Price Pressure Indicators
    df['net_order_flow'] = df['buy_qty'] - df['sell_qty']
    df['normalized_net_flow'] = df['net_order_flow'] / (df['volume'] + 1e-10)
    df['buying_pressure'] = df['buy_qty'] / (df['volume'] + 1e-10)
    df['volume_weighted_buy'] = df['buy_qty'] * df['volume']
    
    # Liquidity Depth Measures
    df['total_depth'] = df['bid_qty'] + df['ask_qty']
    df['depth_imbalance'] = (df['bid_qty'] - df['ask_qty']) / (df['total_depth'] + 1e-10)
    df['relative_spread'] = np.abs(df['bid_qty'] - df['ask_qty']) / (df['total_depth'] + 1e-10)
    df['log_depth'] = np.log1p(df['total_depth'])
    
    # Order Flow Toxicity Proxies
    df['kyle_lambda'] = np.abs(df['net_order_flow']) / (df['volume'] + 1e-10)
    df['flow_toxicity'] = np.abs(df['order_flow_imbalance']) * df['volume']
    df['aggressive_flow_ratio'] = (df['buy_qty'] + df['sell_qty']) / (df['total_depth'] + 1e-10)
    
    # Market Activity Indicators
    df['volume_depth_ratio'] = df['volume'] / (df['total_depth'] + 1e-10)
    df['activity_intensity'] = (df['buy_qty'] + df['sell_qty']) / (df['volume'] + 1e-10)
    df['log_buy_qty'] = np.log1p(df['buy_qty'])
    df['log_sell_qty'] = np.log1p(df['sell_qty'])
    df['log_bid_qty'] = np.log1p(df['bid_qty'])
    df['log_ask_qty'] = np.log1p(df['ask_qty'])
    
    # Microstructure Volatility Proxies
    df['realized_spread_proxy'] = 2 * np.abs(df['net_order_flow']) / (df['volume'] + 1e-10)
    df['price_impact_proxy'] = df['net_order_flow'] / (df['total_depth'] + 1e-10)
    df['quote_volatility_proxy'] = np.abs(df['depth_imbalance'])
    
    # Complex Interaction Terms
    df['flow_depth_interaction'] = df['net_order_flow'] * df['total_depth']
    df['imbalance_volume_interaction'] = df['order_flow_imbalance'] * df['volume']
    df['depth_volume_interaction'] = df['total_depth'] * df['volume']
    df['buy_sell_spread'] = np.abs(df['buy_qty'] - df['sell_qty'])
    df['bid_ask_spread'] = np.abs(df['bid_qty'] - df['ask_qty'])
    
    # Information Asymmetry Measures
    df['trade_informativeness'] = df['net_order_flow'] / (df['bid_qty'] + df['ask_qty'] + 1e-10)
    df['execution_shortfall_proxy'] = df['buy_sell_spread'] / (df['volume'] + 1e-10)
    df['adverse_selection_proxy'] = df['net_order_flow'] / (df['total_depth'] + 1e-10) * df['volume']
    
    # Market Efficiency Indicators
    df['fill_probability'] = df['volume'] / (df['buy_qty'] + df['sell_qty'] + 1e-10)
    df['execution_rate'] = (df['buy_qty'] + df['sell_qty']) / (df['total_depth'] + 1e-10)
    df['market_efficiency'] = df['volume'] / (df['bid_ask_spread'] + 1e-10)
    
    # Non-linear Transformations
    df['sqrt_volume'] = np.sqrt(df['volume'])
    df['sqrt_depth'] = np.sqrt(df['total_depth'])
    df['volume_squared'] = df['volume'] ** 2
    df['imbalance_squared'] = df['order_flow_imbalance'] ** 2
    
    # Relative Measures
    df['bid_ratio'] = df['bid_qty'] / (df['total_depth'] + 1e-10)
    df['ask_ratio'] = df['ask_qty'] / (df['total_depth'] + 1e-10)
    df['buy_ratio'] = df['buy_qty'] / (df['buy_qty'] + df['sell_qty'] + 1e-10)
    df['sell_ratio'] = df['sell_qty'] / (df['buy_qty'] + df['sell_qty'] + 1e-10)
    
    # Market Stress Indicators
    df['liquidity_consumption'] = (df['buy_qty'] + df['sell_qty']) / (df['total_depth'] + 1e-10)
    df['market_stress'] = df['volume'] / (df['total_depth'] + 1e-10) * np.abs(df['order_flow_imbalance'])
    df['depth_depletion'] = df['volume'] / (df['bid_qty'] + df['ask_qty'] + 1e-10)
    
    # Directional Indicators
    df['net_buying_ratio'] = df['net_order_flow'] / (df['volume'] + 1e-10)
    df['directional_volume'] = df['net_order_flow'] * np.log1p(df['volume'])
    df['signed_volume'] = np.sign(df['net_order_flow']) * df['volume']
    
    # Replace infinities and NaNs
    df = df.replace([np.inf, -np.inf], 0).fillna(0)
    
    return df

def create_time_decay_weights(n: int, decay: float = 0.9) -> np.ndarray:
    positions = np.arange(n)
    normalized = positions / (n - 1) if n > 1 else positions
    weights = decay ** (1.0 - normalized)
    return weights * n / weights.sum()

def detect_outliers_and_adjust_weights(X, y, sample_weights, outlier_fraction=0.001, strategy="reduce"):
    """
    Detect outliers based on prediction residuals and adjust their weights.
    
    Strategies:
    - "reduce": Current approach - reduce weights to 0.2-0.8x
    - "remove": Set outlier weights to 0 (effectively removing them)
    - "double": Double the weights of outliers
    - "none": No adjustment
    """
    if strategy == "none":
        return sample_weights, np.zeros(len(y), dtype=bool)
    
    # Ensure we have at least some samples to detect outliers
    n_samples = len(y)
    if n_samples < 100:  # Not enough samples for meaningful outlier detection
        print(f"    Too few samples ({n_samples}) for outlier detection")
        return sample_weights, np.zeros(n_samples, dtype=bool)
    
    # Train a simple model to get residuals
    rf = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42, n_jobs=-1)
    rf.fit(X, y, sample_weight=sample_weights)
    
    # Calculate residuals
    predictions = rf.predict(X)
    residuals = np.abs(y - predictions)
    
    # Find threshold for top outlier_fraction
    # Ensure we have at least 1 outlier
    n_outliers = max(1, int(len(residuals) * outlier_fraction))
    
    # Sort residuals and get threshold
    sorted_residuals = np.sort(residuals)
    threshold = sorted_residuals[-n_outliers] if n_outliers <= len(residuals) else sorted_residuals[-1]
    
    # Create outlier mask
    outlier_mask = residuals >= threshold
    
    # Ensure we have exactly n_outliers (handle ties at threshold)
    if np.sum(outlier_mask) > n_outliers:
        # If we have too many due to ties, randomly select to get exact number
        outlier_indices = np.where(outlier_mask)[0]
        np.random.seed(42)
        selected_indices = np.random.choice(outlier_indices, n_outliers, replace=False)
        outlier_mask = np.zeros(len(y), dtype=bool)
        outlier_mask[selected_indices] = True
    
    # Adjust weights based on strategy
    adjusted_weights = sample_weights.copy()
    
    if outlier_mask.any():
        if strategy == "reduce":
            # Original approach: reduce weights proportionally
            outlier_residuals = residuals[outlier_mask]
            min_outlier_res = outlier_residuals.min()
            max_outlier_res = outlier_residuals.max()
            
            if max_outlier_res > min_outlier_res:
                normalized_residuals = (outlier_residuals - min_outlier_res) / (max_outlier_res - min_outlier_res)
            else:
                normalized_residuals = np.ones_like(outlier_residuals)
            
            weight_factors = 0.8 - 0.6 * normalized_residuals
            adjusted_weights[outlier_mask] *= weight_factors
            
        elif strategy == "remove":
            # Set outlier weights to 0
            adjusted_weights[outlier_mask] = 0
            
        elif strategy == "double":
            # Double the weights of outliers
            adjusted_weights[outlier_mask] *= 2.0
        
        print(f"    Strategy '{strategy}': Adjusted {n_outliers} outliers ({outlier_fraction*100:.1f}% of data)")
    
    return adjusted_weights, outlier_mask

def load_data():
    # Load data with all features available
    all_features = list(set(Config.FEATURES + Config.MLP_FEATURES))
    train_df = pd.read_parquet(Config.TRAIN_PATH, columns=all_features + [Config.LABEL_COLUMN])
    test_df = pd.read_parquet(Config.TEST_PATH, columns=all_features)
    submission_df = pd.read_csv(Config.SUBMISSION_PATH)
    print(f"Loaded data - Train: {train_df.shape}, Test: {test_df.shape}, Submission: {submission_df.shape}")

    # Add features
    train_df = add_features(train_df)
    test_df = add_features(test_df)

    # Update Config.FEATURES with new features
    Config.FEATURES += [
        "log_volume", 'bid_ask_interaction', 'bid_buy_interaction', 'bid_sell_interaction', 
        'ask_buy_interaction', 'ask_sell_interaction', 'net_order_flow', 'normalized_net_flow',
        'buying_pressure', 'volume_weighted_buy', 'total_depth', 'depth_imbalance',
        'relative_spread', 'log_depth', 'kyle_lambda', 'flow_toxicity', 'aggressive_flow_ratio',
        'volume_depth_ratio', 'activity_intensity', 'log_buy_qty', 'log_sell_qty',
        'log_bid_qty', 'log_ask_qty', 'realized_spread_proxy', 'price_impact_proxy',
        'quote_volatility_proxy', 'flow_depth_interaction', 'imbalance_volume_interaction',
        'depth_volume_interaction', 'buy_sell_spread', 'bid_ask_spread', 'trade_informativeness',
        'execution_shortfall_proxy', 'adverse_selection_proxy', 'fill_probability',
        'execution_rate', 'market_efficiency', 'sqrt_volume', 'sqrt_depth', 'volume_squared',
        'imbalance_squared', 'bid_ratio', 'ask_ratio', 'buy_ratio', 'sell_ratio',
        'liquidity_consumption', 'market_stress', 'depth_depletion', 'net_buying_ratio',
        'directional_volume', 'signed_volume'
    ]

    return train_df.reset_index(drop=True), test_df.reset_index(drop=True), submission_df

def get_model_slices(n_samples: int):
    # Original 5 slices
    base_slices = [
        {"name": "full_data", "cutoff": 0, "is_oldest": False, "outlier_adjusted": False},
        {"name": "last_90pct", "cutoff": int(0.10 * n_samples), "is_oldest": False, "outlier_adjusted": False},
        {"name": "last_85pct", "cutoff": int(0.15 * n_samples), "is_oldest": False, "outlier_adjusted": False},
        {"name": "last_80pct", "cutoff": int(0.20 * n_samples), "is_oldest": False, "outlier_adjusted": False},
        {"name": "oldest_25pct", "cutoff": int(0.25 * n_samples), "is_oldest": True, "outlier_adjusted": False},
    ]
    
    # Duplicate slices with outlier adjustment
    outlier_adjusted_slices = []
    for slice_info in base_slices:
        adjusted_slice = slice_info.copy()
        adjusted_slice["name"] = f"{slice_info['name']}_outlier_adj"
        adjusted_slice["outlier_adjusted"] = True
        outlier_adjusted_slices.append(adjusted_slice)
    
    return base_slices + outlier_adjusted_slices

# =========================
# Outlier Analysis Functions
# =========================
def analyze_outliers(train_df):
    """Analyze outliers in the training data"""
    print("\n=== Outlier Analysis ===")
    
    X = train_df[Config.FEATURES].values
    y = train_df[Config.LABEL_COLUMN].values
    
    # Get base weights
    sample_weights = create_time_decay_weights(len(train_df))
    
    # Detect outliers
    _, outlier_mask = detect_outliers_and_adjust_weights(
        X, y, sample_weights, outlier_fraction=Config.OUTLIER_FRACTION, strategy="reduce"
    )
    
    # Analyze outlier characteristics
    outlier_indices = np.where(outlier_mask)[0]
    n_outliers = len(outlier_indices)
    
    print(f"\nTotal outliers detected: {n_outliers} ({n_outliers/len(train_df)*100:.2f}%)")
    
    if n_outliers > 0:
        # Statistical analysis
        outlier_labels = y[outlier_mask]
        normal_labels = y[~outlier_mask]
        
        print(f"\nLabel statistics:")
        print(f"  Normal samples - Mean: {normal_labels.mean():.4f}, Std: {normal_labels.std():.4f}")
        print(f"  Outlier samples - Mean: {outlier_labels.mean():.4f}, Std: {outlier_labels.std():.4f}")
        print(f"  Label range - Normal: [{normal_labels.min():.4f}, {normal_labels.max():.4f}]")
        print(f"  Label range - Outliers: [{outlier_labels.min():.4f}, {outlier_labels.max():.4f}]")
        
        # Feature analysis for outliers
        print(f"\nTop features with extreme values in outliers:")
        feature_names = Config.FEATURES[:20]  # Analyze first 20 features
        outlier_features = train_df.iloc[outlier_indices][feature_names]
        normal_features = train_df.iloc[~outlier_mask][feature_names]
        
        feature_diffs = []
        for feat in feature_names:
            outlier_mean = outlier_features[feat].mean()
            normal_mean = normal_features[feat].mean()
            if normal_mean != 0:
                rel_diff = abs(outlier_mean - normal_mean) / abs(normal_mean)
                feature_diffs.append((feat, rel_diff, outlier_mean, normal_mean))
        
        feature_diffs.sort(key=lambda x: x[1], reverse=True)
        for feat, diff, out_mean, norm_mean in feature_diffs[:10]:
            print(f"  {feat}: {diff*100:.1f}% difference (outlier: {out_mean:.4f}, normal: {norm_mean:.4f})")
    else:
        print("\nNo outliers detected with current threshold. Consider adjusting outlier_fraction.")
    
    return outlier_indices

# =========================
# XGBoost Training with Outlier Strategy Comparison
# =========================
def train_xgboost_with_outlier_comparison(train_df, test_df):
    """Train XGBoost with different outlier handling strategies and compare results"""
    n_samples = len(train_df)
    
    # Store results for each strategy
    strategy_results = {strategy: {"oof_scores": [], "slice_scores": {}} 
                       for strategy in Config.OUTLIER_STRATEGIES}
    
    # For final ensemble
    best_strategy = "reduce"  # Default to current approach
    best_score = -np.inf
    best_oof_preds = None
    best_test_preds = None
    
    for strategy in Config.OUTLIER_STRATEGIES:
        print(f"\n{'='*50}")
        print(f"Testing outlier strategy: {strategy.upper()}")
        print(f"{'='*50}")
        
        # Get model slices for this strategy
        model_slices = get_model_slices(n_samples)
        
        oof_preds = {
            learner["name"]: {s["name"]: np.zeros(n_samples) for s in model_slices}
            for learner in LEARNERS
        }
        test_preds = {
            learner["name"]: {s["name"]: np.zeros(len(test_df)) for s in model_slices}
            for learner in LEARNERS
        }
        
        full_weights = create_time_decay_weights(n_samples)
        kf = KFold(n_splits=Config.N_FOLDS, shuffle=False)
        
        for fold, (train_idx, valid_idx) in enumerate(kf.split(train_df), start=1):
            print(f"\n--- Fold {fold}/{Config.N_FOLDS} ---")
            X_valid = train_df.iloc[valid_idx][Config.FEATURES]
            y_valid = train_df.iloc[valid_idx][Config.LABEL_COLUMN]
            
            for s in model_slices:
                cutoff = s["cutoff"]
                slice_name = s["name"]
                is_oldest = s["is_oldest"]
                outlier_adjusted = s.get("outlier_adjusted", False)
                
                if is_oldest:
                    subset = train_df.iloc[:cutoff].reset_index(drop=True)
                    rel_idx = train_idx[train_idx < cutoff]
                    sw = np.ones(len(rel_idx))
                else:
                    subset = train_df.iloc[cutoff:].reset_index(drop=True)
                    rel_idx = train_idx[train_idx >= cutoff] - cutoff
                    sw = create_time_decay_weights(len(subset))[rel_idx] if cutoff > 0 else full_weights[train_idx]
                
                X_train = subset.iloc[rel_idx][Config.FEATURES]
                y_train = subset.iloc[rel_idx][Config.LABEL_COLUMN]
                
                # Apply outlier strategy if this is an outlier-adjusted slice
                if outlier_adjusted and len(X_train) > 100:
                    sw, _ = detect_outliers_and_adjust_weights(
                        X_train.values, 
                        y_train.values, 
                        sw, 
                        outlier_fraction=Config.OUTLIER_FRACTION,
                        strategy=strategy
                    )
                
                print(f"  Training slice: {slice_name}, samples: {len(X_train)}")
                
                for learner in LEARNERS:
                    model = learner["Estimator"](**learner["params"])
                    model.fit(X_train, y_train, sample_weight=sw, eval_set=[(X_valid, y_valid)], verbose=False)
                    
                    if is_oldest:
                        oof_preds[learner["name"]][slice_name][valid_idx] = model.predict(
                            train_df.iloc[valid_idx][Config.FEATURES]
                        )
                    else:
                        mask = valid_idx >= cutoff
                        if mask.any():
                            idxs = valid_idx[mask]
                            oof_preds[learner["name"]][slice_name][idxs] = model.predict(
                                train_df.iloc[idxs][Config.FEATURES]
                            )
                        if cutoff > 0 and (~mask).any():
                            base_slice_name = slice_name.replace("_outlier_adj", "")
                            if base_slice_name == slice_name:
                                fallback_slice = "full_data"
                            else:
                                fallback_slice = "full_data_outlier_adj"
                            oof_preds[learner["name"]][slice_name][valid_idx[~mask]] = oof_preds[learner["name"]][fallback_slice][
                                valid_idx[~mask]
                            ]
                    
                    test_preds[learner["name"]][slice_name] += model.predict(test_df[Config.FEATURES])
        
        # Normalize test predictions
        for learner_name in test_preds:
            for slice_name in test_preds[learner_name]:
                test_preds[learner_name][slice_name] /= Config.N_FOLDS
        
        # Evaluate this strategy
        learner_name = 'xgb'
        
        # Weights for ensemble
        weights = np.array([
            1.0,   # full_data
            1.0,   # last_90pct
            1.0,   # last_85pct
            1.0,   # last_80pct
            0.25,  # oldest_25pct
            0.9,   # full_data_outlier_adj
            0.9,   # last_90pct_outlier_adj
            0.9,   # last_85pct_outlier_adj
            0.9,   # last_80pct_outlier_adj
            0.2    # oldest_25pct_outlier_adj
        ])
        weights = weights / weights.sum()
        
        oof_weighted = pd.DataFrame(oof_preds[learner_name]).values @ weights
        test_weighted = pd.DataFrame(test_preds[learner_name]).values @ weights
        score_weighted = pearsonr(train_df[Config.LABEL_COLUMN], oof_weighted)[0]
        
        print(f"\n{strategy.upper()} Strategy - Weighted Ensemble Pearson: {score_weighted:.4f}")
        
        # Store individual slice scores
        slice_names = list(oof_preds[learner_name].keys())
        for i, slice_name in enumerate(slice_names):
            score = pearsonr(train_df[Config.LABEL_COLUMN], oof_preds[learner_name][slice_name])[0]
            strategy_results[strategy]["slice_scores"][slice_name] = score
            if "outlier_adj" in slice_name:
                print(f"  {slice_name}: {score:.4f} (weight: {weights[i]:.3f})")
        
        strategy_results[strategy]["oof_scores"].append(score_weighted)
        strategy_results[strategy]["ensemble_score"] = score_weighted
        strategy_results[strategy]["oof_preds"] = oof_weighted
        strategy_results[strategy]["test_preds"] = test_weighted
        
        # Track best strategy
        if score_weighted > best_score:
            best_score = score_weighted
            best_strategy = strategy
            best_oof_preds = oof_preds
            best_test_preds = test_preds
    
    # Print comparison summary
    print(f"\n{'='*50}")
    print("OUTLIER STRATEGY COMPARISON SUMMARY")
    print(f"{'='*50}")
    
    for strategy in Config.OUTLIER_STRATEGIES:
        score = strategy_results[strategy]["ensemble_score"]
        print(f"{strategy.upper()}: {score:.4f} {'← BEST' if strategy == best_strategy else ''}")
    
    # Analyze differences
    print(f"\nRelative performance vs 'reduce' strategy:")
    reduce_score = strategy_results["reduce"]["ensemble_score"]
    for strategy in Config.OUTLIER_STRATEGIES:
        if strategy != "reduce":
            score = strategy_results[strategy]["ensemble_score"]
            diff = (score - reduce_score) / reduce_score * 100
            print(f"  {strategy}: {diff:+.2f}%")
    
    return best_oof_preds, best_test_preds, model_slices, strategy_results, best_strategy

# =========================
# MLP Training (unchanged)
# =========================
def train_mlp(train_df, test_df):
    print("\n=== Training MLP Model ===")
    
    # Hyperparameters
    hparams = {
        "seed": 42,
        "num_epochs": 10,
        "batch_size": 1024 * 8 * 4,
        "learning_rate": 0.001,
        "weight_decay": 1e-3,
        "dropout_rate": 0.6,
        "layers": [len(Config.MLP_FEATURES), 256, 64, 1],
        "hidden_activation": None,
        "activation": "relu",
        "delta": 5,
        "noise_factor": 0.005
    }
    
    set_seed(hparams["seed"])
    
    # Prepare data for MLP
    X_train_full = train_df[Config.MLP_FEATURES].values
    y_train_full = train_df[Config.LABEL_COLUMN].values
    
    # Split for validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_full, y_train_full, test_size=0.2, shuffle=False, random_state=42
    )
    
    # Scale data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(test_df[Config.MLP_FEATURES].values)
    
    # Create dataloaders
    train_loader = get_dataloaders(X_train, y_train, hparams, device, shuffle=True)
    val_loader = get_dataloaders(X_val, y_val, hparams, device, shuffle=False)
    test_loader = get_dataloaders(X_test, None, hparams, device, shuffle=False)
    
    # Initialize model
    model = MLP(
        layers=hparams["layers"],
        dropout_rate=hparams["dropout_rate"],
        activation=hparams["activation"],
        last_activation=hparams["hidden_activation"],
    ).to(device)
    
    criterion = nn.HuberLoss(delta=hparams["delta"], reduction='sum')
    optimizer = optim.Adam(model.parameters(), lr=hparams["learning_rate"], 
                          weight_decay=hparams["weight_decay"])
    
    checkpointer = Checkpointer(path="best_mlp_model.pt")
    
    # Training loop
    num_epochs = hparams["num_epochs"]
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs, targets = inputs.to(device), targets.to(device)
            
            # Add noise for robustness
            inputs = inputs + torch.randn_like(inputs) * hparams["noise_factor"]
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            
        running_loss = running_loss / len(train_loader.dataset)
        print(f"Training Loss: {running_loss:.4f}")

        # Validation phase
        model.eval()
        val_loss = 0.0
        preds = []
        trues = []
        with torch.no_grad():
            for inputs, targets in tqdm(val_loader, desc="Validation"):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)
                preds.append(outputs.cpu().numpy())
                trues.append(targets.cpu().numpy())

        val_loss /= len(val_loader.dataset)
        preds = np.concatenate(preds).flatten()
        trues = np.concatenate(trues).flatten()
        pearson_coef = pearsonr(preds, trues)[0]
        print(f"Validation Pearson Coef: {pearson_coef:.4f} | Loss: {val_loss:.4f}")

        checkpointer(pearson_coef, model)
    
    # Load best model and make predictions
    model = checkpointer.load(model)
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs in tqdm(test_loader, desc="Predicting"):
            inputs = inputs[0].to(device)
            outputs = model(inputs)
            predictions.append(outputs.cpu().numpy())

    predictions = np.concatenate(predictions).flatten()
    
    return predictions

# =========================
# Ensemble & Submission Functions
# =========================
def create_xgboost_submission(train_df, oof_preds, test_preds, submission_df, strategy="reduce"):
    learner_name = 'xgb'
    
    # Weights for 10 slices
    weights = np.array([
        1.0,   # full_data
        1.0,   # last_90pct
        1.0,   # last_85pct
        1.0,   # last_80pct
        0.25,  # oldest_25pct
        0.9,   # full_data_outlier_adj
        0.9,   # last_90pct_outlier_adj
        0.9,   # last_85pct_outlier_adj
        0.9,   # last_80pct_outlier_adj
        0.2    # oldest_25pct_outlier_adj
    ])
    
    # Normalize weights
    weights = weights / weights.sum()

    oof_weighted = pd.DataFrame(oof_preds[learner_name]).values @ weights
    test_weighted = pd.DataFrame(test_preds[learner_name]).values @ weights
    score_weighted = pearsonr(train_df[Config.LABEL_COLUMN], oof_weighted)[0]
    print(f"\n{learner_name.upper()} Weighted Ensemble Pearson: {score_weighted:.4f}")

    # Print individual slice scores and weights for analysis
    print("\nIndividual slice OOF scores and weights:")
    slice_names = list(oof_preds[learner_name].keys())
    for i, slice_name in enumerate(slice_names):
        score = pearsonr(train_df[Config.LABEL_COLUMN], oof_preds[learner_name][slice_name])[0]
        print(f"  {slice_name}: {score:.4f} (weight: {weights[i]:.3f})")

    # Save XGBoost submission
    xgb_submission = submission_df.copy()
    xgb_submission["prediction"] = test_weighted
    xgb_submission.to_csv(f"submission_xgboost_{strategy}.csv", index=False)
    print(f"\nSaved: submission_xgboost_{strategy}.csv")
    
    return test_weighted

def create_ensemble_submission(xgb_predictions, mlp_predictions, submission_df, 
                             xgb_weight=0.9, mlp_weight=0.1, suffix=""):
    # Ensemble predictions
    ensemble_predictions = (xgb_weight * xgb_predictions + 
                          mlp_weight * mlp_predictions)
    
    # Save ensemble submission
    ensemble_submission = submission_df.copy()
    ensemble_submission["prediction"] = ensemble_predictions
    filename = f"submission_ensemble{suffix}.csv"
    ensemble_submission.to_csv(filename, index=False)
    print(f"\nSaved: {filename} (XGBoost: {xgb_weight*100}%, MLP: {mlp_weight*100}%)")
    
    return ensemble_predictions

# =========================
# Main Execution
# =========================
if __name__ == "__main__":
    # Load data
    train_df, test_df, submission_df = load_data()
    
    # Analyze outliers
    outlier_indices = analyze_outliers(train_df)
    
    # Train XGBoost with outlier comparison
    print("\n=== Training XGBoost Models with Outlier Strategy Comparison ===")
    best_oof_preds, best_test_preds, model_slices, strategy_results, best_strategy = \
        train_xgboost_with_outlier_comparison(train_df, test_df)
    
    # Create XGBoost submission with best strategy
    xgb_predictions = create_xgboost_submission(
        train_df, best_oof_preds, best_test_preds, submission_df, strategy=best_strategy
    )
    
    # Train MLP model
    mlp_predictions = train_mlp(train_df, test_df)
    
    # Save MLP submission
    mlp_submission = submission_df.copy()
    mlp_submission["prediction"] = mlp_predictions
    mlp_submission.to_csv("submission_mlp.csv", index=False)
    print("\nSaved: submission_mlp.csv")
    
    # Create ensemble submission
    ensemble_predictions = create_ensemble_submission(
        xgb_predictions, mlp_predictions, submission_df,
        xgb_weight=0.9, mlp_weight=0.1, suffix=f"_{best_strategy}"
    )
    
    # Print final summary
    print("\n" + "="*60)
    print("FINAL SUMMARY")
    print("="*60)
    print(f"\nBest outlier strategy: {best_strategy.upper()}")
    print(f"Best XGBoost CV score: {strategy_results[best_strategy]['ensemble_score']:.4f}")
    
    print("\nStrategy comparison (XGBoost ensemble scores):")
    for strategy in Config.OUTLIER_STRATEGIES:
        score = strategy_results[strategy]["ensemble_score"]
        print(f"  {strategy}: {score:.4f}")
    
    print("\nCreated submission files:")
    print(f"1. submission_xgboost_{best_strategy}.csv - XGBoost with {best_strategy} strategy")
    print(f"2. submission_mlp.csv - MLP only")
    print(f"3. submission_ensemble_{best_strategy}.csv - 90% XGBoost + 10% MLP")
    
    # Show sample predictions
    print("\nSample predictions (first 10 rows):")
    comparison_df = pd.DataFrame({
        'ID': submission_df['ID'][:10],
        'XGBoost': xgb_predictions[:10],
        'MLP': mlp_predictions[:10],
        'Ensemble': ensemble_predictions[:10]
    })
    print(comparison_df)
    
    # Provide recommendations
    print("\n" + "="*60)
    print("RECOMMENDATIONS")
    print("="*60)
    
    reduce_score = strategy_results["reduce"]["ensemble_score"]
    remove_score = strategy_results["remove"]["ensemble_score"]
    double_score = strategy_results["double"]["ensemble_score"]
    none_score = strategy_results["none"]["ensemble_score"]
    
    print("\n1. Outlier Handling Impact:")
    if best_strategy == "reduce":
        print("   ✓ Current approach (reduce weights) is optimal")
    elif best_strategy == "remove":
        print("   ! Removing outliers completely performs better")
        print("   → This suggests outliers are noise rather than informative extremes")
    elif best_strategy == "double":
        print("   ! Doubling outlier weights performs better")
        print("   → This suggests outliers contain valuable signal for extreme movements")
    else:
        print("   ! No outlier adjustment performs better")
        print("   → This suggests the model can handle outliers naturally")
    
    print("\n2. Overfitting Risk Assessment:")
    if remove_score > reduce_score and remove_score > double_score:
        print("   ⚠ Removing outliers improves CV but may increase overfitting risk")
        print("   → Consider using reduce strategy for better generalization")
    elif double_score > reduce_score:
        print("   ✓ Emphasizing outliers improves performance")
        print("   → Model benefits from learning extreme patterns")
    
    print("\n3. Next Steps:")
    print("   • Test different outlier fractions (0.05%, 0.2%, 0.5%)")
    print("   • Try adaptive outlier detection per time slice")
    print("   • Consider feature-specific outlier handling")
    print("   • Monitor LB score vs CV score for overfitting signs")
    
    # Additional insights based on outlier analysis
    if len(outlier_indices) > 0:
        print(f"\n4. Outlier Insights:")
        print(f"   • Detected {len(outlier_indices)} outliers ({len(outlier_indices)/len(train_df)*100:.2f}% of data)")
        print("   • Consider creating synthetic outliers if 'double' strategy works well")
        print("   • Analyze time distribution of outliers for market regime insights")

Using device: cpu
Loaded data - Train: (525887, 32), Test: (538150, 31), Submission: (538150, 2)

=== Outlier Analysis ===
    Strategy 'reduce': Adjusted 525 outliers (0.1% of data)

Total outliers detected: 525 (0.10%)

Label statistics:
  Normal samples - Mean: 0.0379, Std: 0.9730
  Outlier samples - Mean: -1.7577, Std: 8.4272
  Label range - Normal: [-15.8988, 20.7403]
  Label range - Outliers: [-24.4166, 13.1532]

Top features with extreme values in outliers:
  X345: 974.9% difference (outlier: -0.2079, normal: 0.0238)
  X598: 688.8% difference (outlier: -0.1776, normal: 0.0302)
  buy_qty: 367.4% difference (outlier: 613.4614, normal: 131.2453)
  X385: 121.7% difference (outlier: -0.0153, normal: 0.0704)
  X168: 82.7% difference (outlier: 0.0255, normal: 0.1475)
  X603: 82.2% difference (outlier: 0.2858, normal: 0.1568)
  X174: 79.9% difference (outlier: 0.0296, normal: 0.1474)
  X302: 72.7% difference (outlier: 0.0634, normal: 0.2318)
  X415: 61.9% difference (outlier: 0.0684, no

Epoch 1/10: 100%|██████████| 13/13 [00:07<00:00,  1.68it/s]


Training Loss: 16059.1512


Validation: 100%|██████████| 4/4 [00:01<00:00,  2.93it/s]


Validation Pearson Coef: 0.0700 | Loss: 15180.1740
✅ New best model saved with Pearson: 0.0700


...

Training Loss: 14737.5036


Validation: 100%|██████████| 4/4 [00:01<00:00,  2.71it/s]


Validation Pearson Coef: 0.1117 | Loss: 15297.1565
✅ New best model saved with Pearson: 0.1117
Model loaded from best_mlp_model.pt with best Pearson: 0.1117


Predicting: 100%|██████████| 17/17 [00:04<00:00,  3.75it/s]



Saved: submission_mlp.csv

Saved: submission_ensemble_double.csv (XGBoost: 90.0%, MLP: 10.0%)

FINAL SUMMARY

Best outlier strategy: DOUBLE
Best XGBoost CV score: 0.1108

Strategy comparison (XGBoost ensemble scores):
  reduce: 0.1101
  remove: 0.1107
  double: 0.1108
  none: 0.1106

Created submission files:
1. submission_xgboost_double.csv - XGBoost with double strategy
2. submission_mlp.csv - MLP only
3. submission_ensemble_double.csv - 90% XGBoost + 10% MLP

Sample predictions (first 10 rows):
   ID   XGBoost       MLP  Ensemble
0   1  0.035090  0.217608  0.053342
1   2  0.018095 -0.070474  0.009238
2   3  0.136497  0.070964  0.129943
3   4 -0.085066 -0.007894 -0.077348
4   5  0.211597  0.238769  0.214314
5   6 -0.172887 -0.198609 -0.175459
6   7 -0.419656 -0.523997 -0.430090
7   8 -0.154374 -0.422084 -0.181145
8   9  0.222065 -0.204875  0.179371
9  10  0.096913  0.106206  0.097842

RECOMMENDATIONS

1. Outlier Handling Impact:
   ! Doubling outlier weights performs better
   → Thi